<a href="https://colab.research.google.com/github/rajchats2k2/ML/blob/master/Week7Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

tf.random.set_seed(1)

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print(len(vocab))

65


In [0]:
# Create mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding (dim=256)
tf.keras.layers.LSTM (units=1024)
tf.keras.layers.Dense

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
lstm_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, lstm_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units,
                        #return_sequences=True,
                        #stateful=True,
                        #recurrent_initializer='glorot_uniform')),
    tf.keras.layers.LSTM(lstm_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  lstm_units=lstm_units,
  batch_size=BATCH_SIZE)

In [72]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [73]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_4 (Dense)              (64, None, 64)            65600     
_________________________________________________________________
dense_5 (Dense)              (64, None, 65)            4225      
Total params: 5,333,441
Trainable params: 5,333,441
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [75]:
sampled_indices

array([43, 41, 53, 28, 44, 50, 41, 32, 25,  1, 13,  0, 12, 46, 27, 58, 47,
       64, 30, 58, 48, 59, 21, 28, 19,  3, 50, 46, 27, 16, 44, 32, 16, 33,
       53,  6, 64, 26,  0, 47,  9, 33, 54, 18, 10, 36, 64, 40, 39, 10, 43,
       30, 26, 27, 55, 53,  2,  8, 58, 55,  6, 41, 56, 19, 42, 19, 11,  6,
       25, 30, 40, 54, 40, 21, 59, 25, 17, 38, 63, 23, 32, 11, 16,  7, 62,
        1, 26, 61, 42, 40, 59,  2, 51, 31, 35,  5, 58, 48, 10, 61])

In [76]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'nd so are like to do.\n\nCOMINIUS:\nThat is the way to lay the city flat;\nTo bring the roof to the foun'

Next Char Predictions: 
 "ecoPflcTM A\n?hOtizRtjuIPG$lhODfTDUo,zN\ni3UpF:Xzba:eRNOqo!.tq,crGdG;,MRbpbIuMEZyKT;D-x Nwdbu!mSW'tj:w"


In [77]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173914


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [81]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 31s 179ms/step - loss: 2.8628
Epoch 2/10
172/172 [==============================] - 29s 170ms/step - loss: 2.1362
Epoch 3/10
172/172 [==============================] - 29s 171ms/step - loss: 1.8581
Epoch 4/10
172/172 [==============================] - 30s 172ms/step - loss: 1.6845
Epoch 5/10
172/172 [==============================] - 29s 171ms/step - loss: 1.5767
Epoch 6/10
172/172 [==============================] - 29s 171ms/step - loss: 1.5042
Epoch 7/10
172/172 [==============================] - 29s 171ms/step - loss: 1.4521
Epoch 8/10
172/172 [==============================] - 29s 170ms/step - loss: 1.4136
Epoch 9/10
172/172 [==============================] - 29s 170ms/step - loss: 1.3810
Epoch 10/10
172/172 [==============================] - 29s 171ms/step - loss: 1.3547


In [31]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, lstm_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: th shall sole life
Are you calling you to saily nor condemn'd
Like and the haterful man a trait i' the way
To most put forth their flain and spots,
I'll speak, lest flouts, that Richmond is it were most
seeming, that is bother, then at last, thougarts the rock.

CAPULET:
Reconditual members.

ARCELIS:
Let's fear:
Our debotced men?

MAny yields; commend
overshis a milk, I'll give you admits might lips
To use the hand that now old Oxford is
rebuke a servant better.

AUFIDIUS:
I would, I fly the Lady Grey.

KING EDWARD IV:
Gran, that is here hath some touches as this:
For my rut-le, if we banish you?

Citizens:
I wishem too, but what hands whose louds hath seed
That lands ather out to charm and pull the
rust we give you whats, I here provided all the envy:

G HENRY VI
Come away or, hoor.

BAPTISTA:
Why, come, here, my master, nor I my time?
Tell me thy mother flattering both nexts in me have
Divided to content thee fuction from his beh,
And give the man: all Romeo is no sacred grea